In [80]:
# Import Library
import pandas as pd
import re


In [127]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import TimeoutException



In [134]:
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=OFF&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Arian FosterA. Foster,IND@HOU,41.0,0,0,0,0,0,0,33,...,0,1,7,0,0,0,0,HOU,2010.0,1.0
1,Peyton ManningP. Manning,IND@HOU,35.0,57,40,433,3,0,0,0,...,0,0,0,0,0,0,0,IND,2010.0,1.0
2,Matt ForteM. Forte,DET@CHI,31.0,0,0,0,0,0,0,17,...,0,7,151,2,0,1,0,CHI,2010.0,1.0
3,Tom BradyT. Brady,CIN@NE,28.0,35,25,258,3,0,0,0,...,0,0,0,0,0,0,0,NE,2010.0,1.0
4,Chris JohnsonC. Johnson,OAK@TEN,26.0,0,0,0,0,0,0,27,...,0,4,8,0,0,0,0,TEN,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56527,Dede WestbrookD. Westbrook,MIN@GB,0.0,0,0,0,0,0,0,0,...,0,1,4,0,0,0,0,MIN,2021.0,17.0
56528,Jonathan WilliamsJ. Williams,PHI@WAS,0.0,0,0,0,0,0,0,2,...,0,2,7,0,0,0,0,WAS,2021.0,17.0
56529,Olamide ZaccheausO. Zaccheaus,ATL@BUF,0.0,0,0,0,0,0,0,0,...,0,2,8,0,0,0,0,ATL,2021.0,17.0
56530,Diontae SpencerD. Spencer,DEN@LAC,-1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,DEN,2021.0,17.0


In [135]:
df.to_csv('data_raw/weekly_offense.csv')

In [139]:
df=pd.DataFrame(columns=['Player', 'Game', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=K&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        #result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df

,Player,Game,Pts*,XPA,XPM,FGA,FGM,50+,team,season,week
0,John KasayJ. Kasay,CAR@NYG,12.0,1,1,3,3,1,CAR,2010.0,1.0
1,Mason CrosbyM. Crosby,GB@PHI,11.0,3,3,2,2,1,GB,2010.0,1.0
2,Jeff ReedJ. Reed,ATL@PIT,11.0,0,0,5,3,1,ATL,2010.0,1.0
3,Neil RackersN. Rackers,IND@HOU,10.0,4,4,2,2,0,HOU,2010.0,1.0
4,Matt BryantM. Bryant,ATL@PIT,9.0,0,0,4,3,0,ATL,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6068,Jason SandersJ. Sanders,MIA@TEN,3.0,0,0,2,1,0,MIA,2021.0,17.0
6069,Matt GayM. Gay,LA@BAL,2.0,2,2,1,0,0,LA,2021.0,17.0
6070,Chase McLaughlinC. McLaughlin,CLE@PIT,2.0,2,2,0,0,0,CLE,2021.0,17.0
6071,Greg ZuerleinG. Zuerlein,ARI@DAL,2.0,2,2,1,0,0,DAL,2021.0,17.0


In [140]:
df.to_csv('data_raw/weekly_kicker.csv')

In [144]:
df=pd.DataFrame(columns=['Team', 'Game', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds', 'RushYds', 'TotYds'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        #result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df



,Team,Game,Pts*,Sack,Int,Saf,FR,Blk,TD,PA,PassYds,RushYds,TotYds,team,season,week
0,Seattle SeahawksSEA,SF@SEA,22.0,2.0,2,0,0,0,1,6,214,49,263,SEA,2010.0,1.0
1,Arizona CardinalsARI,ARI@STL,19.0,2.0,3,0,1,1,0,13,240,85,325,ARI,2010.0,1.0
2,New York GiantsNYG,CAR@NYG,18.0,4.0,3,0,2,0,0,18,148,89,237,NYG,2010.0,1.0
3,New England PatriotsNE,CIN@NE,18.0,1.0,1,0,1,0,2,24,341,87,428,NE,2010.0,1.0
4,Detroit LionsDET,DET@CHI,16.0,4.0,1,0,3,0,0,19,362,101,463,DET,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,Miami DolphinsMIA,MIA@TEN,1.0,1.0,0,0,0,0,0,34,110,198,308,MIA,2021.0,17.0
6140,Detroit LionsDET,DET@SEA,1.0,1.0,0,0,0,0,0,51,232,265,497,DET,2021.0,17.0
6141,Denver BroncosDEN,DEN@LAC,0.0,0.0,0,0,0,0,0,34,237,104,341,DEN,2021.0,17.0
6142,Minnesota VikingsMIN,MIN@GB,0.0,0.0,0,0,0,0,0,37,307,174,481,MIN,2021.0,17.0


In [145]:
df.to_csv('data_raw/weekly_defense.csv')